In [19]:
import pandas as pd
import requests
from pandas import json_normalize
import json
import warnings

In [25]:
warnings.filterwarnings("ignore")

In [44]:
# https://admin-api.docs.keitaro.io/#tag/Clicks/paths/~1clicks~1log/post

payload = json.dumps({
  "limit": 10, # change to 0 for pulling all clicks history
  "offset": 0,
  "filters": [
    {
      "name": "campaign_group_id",
      "operator": "EQUALS",
      "expression": 00 #add group id you want to filter on
    }
  ],
  "range": {
       "to": "2024-04-09" # pulls all the data till the specified date
  #    "interval": "yesterday"  # uncomment for pulling data for yesterday only
    }
})

clicks = requests.post(
    'http://keitaro-example-server.com/admin_api/v1/clicks/log', headers={'Api-Key': ''}, # add your actual keitaro server address
    data=payload
)

In [45]:
clicks

<Response [200]>

In [46]:
df_clicks = json_normalize(clicks.json()['rows'])
df_clicks_filtered = df_clicks[['campaign_id', 'campaign', 'click_id', 'sub_id', 'sub_id_1', \
                                'sub_id_2', 'sub_id_3', 'sub_id_4', 'sub_id_5', 'datetime']]
df_clicks_filtered['datetime'] = pd.to_datetime(df_clicks_filtered['datetime'])

In [ ]:
import clickhouse_connect

# https://clickhouse-driver.readthedocs.io/en/latest/quickstart.html#selecting-data

client = clickhouse_connect.get_client(host='', port=8123, username='', password='')

In [213]:
# https://clickhouse.com/docs/en/integrations/python#client-insert-method
client.insert_df(table='scheme.keitaro_clicks', df=df_clicks_filtered)